In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
# 1. Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Using device: NVIDIA GeForce RTX 2070 SUPER


In [3]:
# 2. Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)

# Class names for CIFAR-10
classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 3. Define CNN Model with Configurable Kernel and Pooling
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type="max"):
        super(CNN, self).__init__()
        pooling_layer = nn.MaxPool2d if pooling_type == "max" else nn.AvgPool2d

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            pooling_layer(2, 2),
            nn.Conv2d(32, 64, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            pooling_layer(2, 2),
            nn.Conv2d(64, 128, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            pooling_layer(2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [5]:
# 4. Define Training and Testing Functions
def train_model(model, criterion, optimizer, train_loader, scheduler=None):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if scheduler:
        scheduler.step()
    return running_loss / len(train_loader)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [6]:
# 5. Experiment Configurations
kernel_sizes = [3, 5, 7]
pooling_types = ["max", "avg"]
num_epochs_list = [5, 50, 100, 250, 350]
optimizers = {"SGD": optim.SGD, "RMSProp": optim.RMSprop, "Adam": optim.Adam}

# Store Results
results = {}

In [ ]:
# 6. Experiment Loop
for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for opt_name, opt_class in optimizers.items():
            print(f"Kernel: {kernel_size}, Pooling: {pooling_type}, Optimizer: {opt_name}")
            
            # Initialize model, loss function, and optimizer
            model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = opt_class(model.parameters(), lr=0.001)
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

            epoch_accuracies = []

            for num_epochs in num_epochs_list:
                print(f"Training for {num_epochs} epochs")

                # Early Stopping Setup
                early_stop_counter = 0
                best_accuracy = 0
                patience = 5

                for epoch in range(num_epochs):
                    train_loss = train_model(model, criterion, optimizer, train_loader, scheduler)
                    test_accuracy = evaluate_model(model, test_loader)

                    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {train_loss:.4f}, Accuracy: {test_accuracy:.4f}")

                    if test_accuracy > best_accuracy:
                        best_accuracy = test_accuracy
                        early_stop_counter = 0
                    else:
                        early_stop_counter += 1

                    if early_stop_counter >= patience:
                        print("Early stopping triggered.")
                        break

                epoch_accuracies.append(best_accuracy)

            results[(kernel_size, pooling_type, opt_name)] = epoch_accuracies

Kernel: 3, Pooling: max, Optimizer: SGD
Training for 5 epochs


100%|██████████| 49/49 [00:19<00:00,  2.45it/s]


Epoch 1/5 - Loss: 2.3027, Accuracy: 0.1042


100%|██████████| 49/49 [00:18<00:00,  2.60it/s]


Epoch 2/5 - Loss: 2.3025, Accuracy: 0.1042


...

100%|██████████| 49/49 [00:18<00:00,  2.68it/s]


Epoch 5/250 - Loss: 0.1008, Accuracy: 0.7459


100%|██████████| 49/49 [00:18<00:00,  2.68it/s]


Epoch 6/250 - Loss: 0.0967, Accuracy: 0.7463
Early stopping triggered.
Training for 350 epochs


100%|██████████| 49/49 [00:19<00:00, 16.72it/s]

In [ ]:
# 7. Visualize Results
for (kernel_size, pooling_type, opt_name), accuracies in results.items():
    plt.plot(num_epochs_list[:len(accuracies)], accuracies, label=f"{kernel_size}-{pooling_type}-{opt_name}")

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Experiment Results")
plt.legend()
plt.show()